In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

In [ ]:
batch_size = 32
target_size = (256, 256)
folder_name = "testcovid"
directory = f"../Datasets/{folder_name}/Splitted_Dataset"

train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=50,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1/255.)
# validation_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
    directory=f"{directory}/train",
    batch_size=batch_size,
    class_mode='categorical',
    target_size=target_size,
    subset="training",
)

test_generator = test_datagen.flow_from_directory(
    directory=f"{directory}/test",
    batch_size=batch_size,
    class_mode="categorical",
    target_size=target_size,
)

validation_generator = train_datagen.flow_from_directory(
    # directory=f"{directory}/val",
    directory=f"{directory}/train",
    batch_size=batch_size,
    class_mode="categorical",
    target_size=target_size,
    subset="validation",
)

# DenseNet Model

In [ ]:
DenseNetBase = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

layer = DenseNetBase.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu', kernel_initializer='he_normal')(layer)

DenseNetPreds = Dense(2, activation='softmax')(layer)

DenseNetModel = Model(inputs=DenseNetBase.input, outputs=DenseNetPreds)

print(f"Toplam DenseNet katman sayısı: {len(DenseNetModel.layers)}")

In [ ]:
for layer in DenseNetBase.layers:
    layer.trainable = False

for layer in DenseNetBase.layers[-100:]:
    layer.trainable = True

DenseNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
DenseNetModel.summary()

In [ ]:
DenseNetEpochs = 50

DenseNetHistory = DenseNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=DenseNetEpochs
)

In [ ]:
"""
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="max",
    baseline=None,
    restore_best_weights=True,
)
DenseNetHistory2 = DenseNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=10, callbacks=[early_stopping]
)
"""

In [ ]:
DenseNetModel.evaluate(test_generator)

# ResNet

In [ ]:
ResNetBase = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

layer2 = ResNetBase.output

layer2 = GlobalAveragePooling2D()(layer2)

layer2 = Dense(1024, activation='relu')(layer2)

ResNetPreds = Dense(2, activation='softmax')(layer2)

ResNetModel = Model(inputs=ResNetBase.input, outputs=ResNetPreds)

print(f"Toplam ResNet katman sayısı: {len(ResNetModel.layers)}")

In [ ]:
for layer in ResNetBase.layers:
    layer.trainable = False

for layer in ResNetBase.layers[-30:]:
    layer.trainable = True

ResNetModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

ResNetModel.summary()

In [ ]:
ResNetEpochs = 50

ResNetHistory = ResNetModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=ResNetEpochs
)

In [ ]:
ResNetModel.evaluate(test_generator)

# Xception

In [ ]:
XceptionBase = Xception(weights='imagenet', include_top=False, input_shape=(256,256,3))

layer3 = XceptionBase.output
layer3 = GlobalAveragePooling2D()(layer3)

layer3 = Dense(128, activation='relu')(layer3)

XceptionPreds = Dense(2, activation='softmax')(layer3)

XceptionModel = Model(inputs=XceptionBase.input, outputs=XceptionPreds)

print(f"Xception katman sayısı: {len(XceptionModel.layers)}")

In [ ]:
for layer in XceptionBase.layers:
    layer.trainable = False

for layer in XceptionBase.layers[-20:]:
    layer.trainable = True

XceptionModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
XceptionModel.summary()

In [ ]:
XceptionEpochs = 50

XceptionHistory = XceptionModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=XceptionEpochs
)

In [ ]:
XceptionModel.evaluate(test_generator)

# VGG16

In [ ]:
VGG16Base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

layer4 = VGG16Base.output
layer4 = GlobalAveragePooling2D()(layer4)
layer4 = Dense(1024, activation='relu')(layer4)
VGG16Preds = Dense(2, activation='softmax')(layer4)

VGG16Model = Model(inputs=VGG16Base.input, outputs=VGG16Preds)
print(f"Toplam ResNet katman sayısı: {len(VGG16Model.layers)}")

In [ ]:
for layer in VGG16Base.layers:
    layer.trainable = False

for layer in VGG16Base.layers[-7:]:
    layer.trainable = True

VGG16Model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
VGG16Model.summary()

In [ ]:
VGG16Epochs = 50

VGG16History = VGG16Model.fit(
    x=train_generator, validation_data=validation_generator, epochs=VGG16Epochs
)

In [ ]:
VGG16Model.evaluate(test_generator)

# InceptionV3

In [ ]:
InceptionBase = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

layer5 = InceptionBase.output
layer5 = GlobalAveragePooling2D()(layer5)
layer5 = Dense(1024, activation='relu')(layer5)
InceptionPreds = Dense(2, activation='softmax')(layer5)

InceptionModel = Model(inputs=InceptionBase.input, outputs=InceptionPreds)
print(f"Toplam ResNet katman sayısı: {len(InceptionModel.layers)}")

In [ ]:
for layer in InceptionBase.layers:
    layer.trainable = False

for layer in InceptionBase.layers[-100:]:
    layer.trainable = True

InceptionModel.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
InceptionModel.summary()

In [ ]:
InceptionEpochs = 50

InceptionHistory = InceptionModel.fit(
    x=train_generator, validation_data=validation_generator, epochs=InceptionEpochs
)

In [ ]:
InceptionModel.evaluate(test_generator)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 30))

axs[0, 0].set_title("DenseNet")
axs[0, 1].set_title("DenseNet")
axs[1, 0].set_title("ResNet")
axs[1, 1].set_title("ResNet")
axs[2, 0].set_title("Xception")
axs[2, 1].set_title("Xception")
axs[3, 0].set_title("VGG16")
axs[3, 1].set_title("VGG16")
axs[4, 0].set_title("Inception")
axs[4, 1].set_title("Inception")

axs[0, 0].set_xlabel("Epoch")
axs[0, 1].set_xlabel("Epoch")
axs[1, 0].set_xlabel("Epoch")
axs[1, 1].set_xlabel("Epoch")
axs[2, 0].set_xlabel("Epoch")
axs[2, 1].set_xlabel("Epoch")
axs[3, 0].set_xlabel("Epoch")
axs[3, 1].set_xlabel("Epoch")
axs[4, 0].set_xlabel("Epoch")
axs[4, 1].set_xlabel("Epoch")

axs[0, 0].plot(DenseNetHistory.history['accuracy'], label='DenseNet Training Accuracy')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].legend()
axs[0, 0].plot(DenseNetHistory.history['val_accuracy'], label='DenseNet Validation Accuracy')
axs[0, 0].set_title('Validation Accuracy')
axs[0, 0].legend()

axs[0, 1].plot(DenseNetHistory.history['loss'], label='DenseNet Training Loss')
axs[0, 1].set_title('Training Loss')
axs[0, 1].legend()
axs[0, 1].plot(DenseNetHistory.history['val_loss'], label='DenseNet Validation Loss')
axs[0, 1].set_title('Validation Loss')
axs[0, 1].legend()

axs[1, 0].plot(ResNetHistory.history['accuracy'], label='ResNet Training Accuracy')
axs[1, 0].set_title('Training Accuracy')
axs[1, 0].legend()
axs[1, 0].plot(ResNetHistory.history['val_accuracy'], label='ResNet Validation Accuracy')
axs[1, 0].set_title('Validation Accuracy')
axs[1, 0].legend()

axs[1, 1].plot(ResNetHistory.history['loss'], label='ResNet Training Loss')
axs[1, 1].set_title('Training Loss')
axs[1, 1].legend()
axs[1, 1].plot(ResNetHistory.history['val_loss'], label='ResNet Validation Loss')
axs[1, 1].set_title('Validation Loss')
axs[1, 1].legend()

axs[2, 0].plot(XceptionHistory.history['accuracy'], label='Xception Training Accuracy')
axs[2, 0].set_title('Training Accuracy')
axs[2, 0].legend()
axs[2, 0].plot(XceptionHistory.history['val_accuracy'], label='Xception Validation Accuracy')
axs[2, 0].set_title('Validation Accuracy')
axs[2, 0].legend()

axs[2, 1].plot(XceptionHistory.history['loss'], label='Xception Training Loss')
axs[2, 1].set_title('Training Loss')
axs[2, 1].legend()
axs[2, 1].plot(XceptionHistory.history['val_loss'], label='Xception Validation Loss')
axs[2, 1].set_title('Validation Loss')
axs[2, 1].legend()

axs[3, 0].plot(VGG16History.history['accuracy'], label='VGG16 Training Accuracy')
axs[3, 0].set_title('Training Accuracy')
axs[3, 0].legend()
axs[3, 0].plot(VGG16History.history['val_accuracy'], label='VGG16 Validation Accuracy')
axs[3, 0].set_title('Validation Accuracy')
axs[3, 0].legend()

axs[3, 1].plot(VGG16History.history['loss'], label='VGG16 Training Loss')
axs[3, 1].set_title('Training Loss')
axs[3, 1].legend()
axs[3, 1].plot(VGG16History.history['val_loss'], label='VGG16 Validation Loss')
axs[3, 1].set_title('Validation Loss')
axs[3, 1].legend()

axs[4, 0].plot(InceptionHistory.history['accuracy'], label='Inception Training Accuracy')
axs[4, 0].set_title('Training Accuracy')
axs[4, 0].legend()
axs[4, 0].plot(InceptionHistory.history['val_accuracy'], label='Inception Validation Accuracy')
axs[4, 0].set_title('Validation Accuracy')
axs[4, 0].legend()

axs[4, 1].plot(InceptionHistory.history['loss'], label='Inception Training Loss')
axs[4, 1].set_title('Training Loss')
axs[4, 1].legend()
axs[4, 1].plot(InceptionHistory.history['val_loss'], label='Inception Validation Loss')
axs[4, 1].set_title('Validation Loss')
axs[4, 1].legend()

plt.tight_layout()
plt.show()


"""

# plt.suptitle("DenseNet - 100 Katman, ResNet - 10 Katman, Xception - 35 Katman Açık, AdaDelta Optimizer, Özel Epoch")

axs[0, 0].plot(DenseNetHistory.history['accuracy'], label='DenseNet Training Accuracy')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].legend()
axs[0, 0].plot(DenseNetHistory.history['val_accuracy'], label='DenseNet Validation Accuracy')
axs[0, 0].set_title('Validation Accuracy')
axs[0, 0].legend()

axs[0, 1].plot(DenseNetHistory.history['loss'], label='DenseNet Training Loss')
axs[0, 1].set_title('Training Loss')
axs[0, 1].legend()
axs[0, 1].plot(DenseNetHistory.history['val_loss'], label='DenseNet Validation Loss')
axs[0, 1].set_title('Validation Loss')
axs[0, 1].legend()

axs[1, 0].plot(XceptionHistory.history['accuracy'], label='ResNet Training Accuracy')
axs[1, 0].set_title('Training Accuracy')
axs[1, 0].legend()
axs[1, 0].plot(XceptionHistory.history['val_accuracy'], label='ResNet Validation Accuracy')
axs[1, 0].set_title('Validation Accuracy')
axs[1, 0].legend()

axs[1, 1].plot(XceptionHistory.history['loss'], label='ResNet Training Loss')
axs[1, 1].set_title('Training Loss')
axs[1, 1].legend()
axs[1, 1].plot(XceptionHistory.history['val_loss'], label='ResNet Validation Loss')
axs[1, 1].set_title('Validation Loss')
axs[1, 1].legend()

axs[2, 0].plot(InceptionHistory.history['accuracy'], label='ResNet Training Accuracy')
axs[2, 0].set_title('Training Accuracy')
axs[2, 0].legend()
axs[2, 0].plot(InceptionHistory.history['val_accuracy'], label='ResNet Validation Accuracy')
axs[2, 0].set_title('Validation Accuracy')
axs[2, 0].legend()

axs[2, 1].plot(InceptionHistory.history['loss'], label='ResNet Training Loss')
axs[2, 1].set_title('Training Loss')
axs[2, 1].legend()
axs[2, 1].plot(InceptionHistory.history['val_loss'], label='ResNet Validation Loss')
axs[2, 1].set_title('Validation Loss')
axs[2, 1].legend()

plt.tight_layout()
plt.show()"""

In [ ]:
import os
from datetime import datetime
from nbconvert import HTMLExporter

folder_path = f"..\\Results\\{folder_name}"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

current_time = datetime.now().strftime("%H.%M_%d.%m.%Y")

html_filename = f"{current_time}.html"
html_file_path = os.path.join(folder_path, html_filename)

notebook_filename = "main_project.ipynb"
exporter = HTMLExporter()
output, _ = exporter.from_filename(notebook_filename)

with open(html_file_path, 'w', encoding='utf-8') as f:
    f.write(output)

print(f"Notebook saved to {folder_path} as {html_filename}")